In [4]:
import pandas as pd
import numpy as np
from usfull_tools import load_DS
from catboost import Pool, CatBoostRegressor, CatBoostClassifier
from sklearn.model_selection import train_test_split
pd.options.display.max_columns = None
%matplotlib inline

from set_vars import KAGGLE_PREFIX, debug_mode, KAGGLE_DIR, target_column, target_type

train, test = load_DS(debug_mode, KAGGLE_DIR, KAGGLE_PREFIX, '_prepare.csv')
del test

X_train, X_test, y_train, y_test = train_test_split(train[train.columns.drop(target_column)], train[target_column], 
                                                    test_size=0.3, random_state=42)
X_train.shape

(623, 449)

## Column prefixes:
-  _nan - original columns with Nan
-  _nan_median, _nan_min, _nan_max, _nan0 - nan replaced by median
-  _idxmax - nan replaced by most frequent
-  _isnull - is null flag
-  _dummie - dumnie for categorical column

In [27]:
if target_type=='binary':
    loss_function = 'Logloss:border=0.5'
    model = CatBoostClassifier(iterations=20, depth=2, learning_rate=0.1, loss_function=loss_function, eval_metric='Accuracy')
    params = ''
elif target_type=='interval':
    model = CatBoostRegressor()
    params = ''
    
#https://tech.yandex.com/catboost/doc/dg/concepts/python-reference_parameters-list-docpage/#python-reference_parameters-list
#clf = CatBoostRegressor(iterations=60, depth=2, learning_rate=0.1, loss_function=loss_function)

# Для CatBoost требуется явно указывать категориальные переменные
i=0
cat_features = []
for column in X_train.columns:
    if X_train[column].dtype == 'object': cat_features.append(i)
    i +=1

model.fit(X_train, y_train, cat_features)

from sklearn.metrics import mean_absolute_error, accuracy_score
print("accuracy: %.3f"
      % accuracy_score(model.predict(X_test), y_test))



feature_importance = pd.DataFrame({'columns': X_test.columns,
                                   'importance': model.get_feature_importance()})


# submit['winPlacePercPredict'] = clf.predict(submit[used_cols])
# submit['matchRank'] = submit.groupby('matchId')['winPlacePercPredict'].rank()
# match_players = submit.groupby('matchId')['Id'].count()
# match_step = 1/(match_players+1)

# match_stat = pd.DataFrame({'matchId': match_players.index,
#                            'players': match_players.values,
#                            'matchStep': match_step.values})

# sub = pd.merge(submit, match_stat, on='matchId',how='inner')
# sub.sort_values(['matchId', 'winPlacePercPredict'], inplace=True)
# sub['winPlacePerc'] = (sub['matchRank']-1)*sub['matchStep']
# sub.loc[sub.isnull().any(axis=1), ['winPlacePerc']]=1


# sub.to_csv('submission.csv', columns=['Id', 'winPlacePerc'], index=False, sep=',', decimal='.')

0:	learn: 0.7897271	total: 29ms	remaining: 551ms
1:	learn: 0.7800963	total: 54ms	remaining: 486ms
2:	learn: 0.7897271	total: 80.3ms	remaining: 455ms
3:	learn: 0.7897271	total: 104ms	remaining: 417ms
4:	learn: 0.7897271	total: 129ms	remaining: 386ms
5:	learn: 0.7897271	total: 154ms	remaining: 360ms
6:	learn: 0.7849117	total: 177ms	remaining: 328ms
7:	learn: 0.7897271	total: 202ms	remaining: 303ms
8:	learn: 0.7897271	total: 224ms	remaining: 273ms
9:	learn: 0.7945425	total: 255ms	remaining: 255ms
10:	learn: 0.7897271	total: 287ms	remaining: 235ms
11:	learn: 0.8025682	total: 310ms	remaining: 207ms
12:	learn: 0.7977528	total: 337ms	remaining: 181ms
13:	learn: 0.8057785	total: 366ms	remaining: 157ms
14:	learn: 0.8154093	total: 391ms	remaining: 130ms
15:	learn: 0.8170144	total: 415ms	remaining: 104ms
16:	learn: 0.8138042	total: 454ms	remaining: 80.1ms
17:	learn: 0.8154093	total: 484ms	remaining: 53.7ms
18:	learn: 0.8186196	total: 513ms	remaining: 27ms
19:	learn: 0.8234350	total: 538ms	remaini

In [31]:
feature_importance.sort_values('importance', ascending = False)

,columns,importance
276,mult_Fare_nan_median_with_Sex_dummie_female,15.990850
224,mult_Age_nan_max_with_Sex_dummie_male,12.491109
52,mult_Pclass_with_Sex_dummie_female,11.464283
361,sum_Cabin_isnull_with_Sex_dummie_male,8.044539
59,sum_Pclass_with_Embarked_nan_dummie_Q,6.596733
20,Sex_dummie_female,6.576230
21,Sex_dummie_male,5.780120
162,mult_Age_nan_median_with_Sex_dummie_male,4.751471
53,sum_Pclass_with_Sex_dummie_male,4.735904
61,sum_Pclass_with_Embarked_nan_dummie_S,3.206820


In [25]:
X_train.shape

(623, 449)